## Basket Analysis of Products

In [2]:
import mysql.connector
import pymysql
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules

### Creating the connection to MySQL and reading data from the database

In [4]:
# Encode the password to escape special characters
password = quote_plus('pwd@mysql')

# Connect to MySQL database
connection = create_engine(f'mysql+pymysql://<username>:{password}@localhost:<port>/<dbname>')

# Load sales data
sales_data = pd.read_sql('SELECT * FROM purchases', con=connection)

In [89]:
# Create DataFrame
df = pd.DataFrame(sales_data)

## Data Pre-processing

In [90]:
# Remove the commas in product_name
df['product_name'] = df['product_name'].apply(lambda x: x.replace(',', ''))

#### Feature Engineering:
Inorder to perform basket data analysis we need to know what are the items that were bought togther, since the data doesn't have a order id which can help us in understanding which items were bought together we use the survey_response_id and order_date to achieve the same. We concat survey_response_id and order_date and call it as transaction_id, later we group the purchases based on the transaction_id. Since for a given survey_response_id we are grouping the purchases based on the date purchased we can say that a specific customer purchased the items together.

**Challenge:** Out of 1.6M purchase records there are over 750K unique transaction ids and over 784K unique product names. All of this is actual purchase data collected from anonymous Amazon customers by Harvard for research purposes for the period late 2018 to early 2023. Amazon has millions of products, in order to perform basket analysis that yields decent metrics on millions of products we need 10s or 100s of millions of transactions. Since we don't have the right proportinate data i.e., the difference between the number of unique transaction ids and product names is significantly small and since we perform one-hot encoding on product names we would have over 784K columns and with such dimensionality it is nearly impossible to perform basket analysis with good support, lift and confidence.

**Solution:** The solution to this is straightforward and simple, we just sample the data. We sample the data by taking the top 2000 most frequent transaction ids, sampling the data this way will allow us to capture more items that were bought togther and then the products that will be in cosideration would be only those that are part of these 2000 transaction ids. The proportion of number of unique transaction ids and product names is much better in this scenario, allowing us to perform basket analysis with way better support, lift and confidence.

We then create a basket by grouping products into a one-hot encoded matrix.

In [6]:
# Combine user_id and order_date to create a transaction_id
df['transaction_id'] = df['survey_response_id'].astype(str) + '_' + df['order_date'].astype(str)

# Step 1: Get the top 2000 most frequent transaction IDs
top_1500_ids = df['transaction_id'].value_counts().head(2000).index

# Step 2: Filter the dataframe to keep only these transactions
filtered_df = df[df['transaction_id'].isin(top_1500_ids)]

# Step 3: Group products per transaction_id into a one-hot encoded matrix
basket = filtered_df.groupby(['transaction_id', 'product_name']).size().unstack().fillna(0)

# Convert counts to binary (1/0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

/var/folders/gk/st927tjd7x52sv1tg0v35fvm0000gn/T/ipykernel_5040/2149483207.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)  # Convert counts to binary (1/0)


In [81]:
unique_products = len(filtered_df['product_name'].value_counts())
print("Number of unique products in the basket:", unique_products)

Number of unique products in the basket: 28995


## Modeling

We now apply the apriori algirithm on the basket and obtain the frequent itemsets. Frequent itemsets are sets of items that are bought togther with a specific amount of support, in this case we set the support to **0.01**. We then apply the associal rules to the frequent itemsets to capture the support, confidence, lift and others among the itemsets. Here the main metric we use is **lift** and we set the threshold limit to **1.0** for lift.

In [95]:
# Apply Apriori algorithm
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# Generate Association Rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules


/opt/homebrew/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(365 by Whole Foods Market Chip Potato Rippled...,(365 by Whole Foods Market Pretzel Twists Mini...,0.0105,0.0115,0.0100,0.952381,82.815735,0.009879,20.758500,0.998408
1,(365 by Whole Foods Market Pretzel Twists Mini...,(365 by Whole Foods Market Chip Potato Rippled...,0.0115,0.0105,0.0100,0.869565,82.815735,0.009879,7.586167,0.999418
2,(Icelandic Provisions 5.3oz Traditional Skyr Y...,(365 by Whole Foods Market Pretzel Twists Mini...,0.0165,0.0115,0.0105,0.636364,55.335968,0.010310,2.718375,0.998402
3,(365 by Whole Foods Market Pretzel Twists Mini...,(Icelandic Provisions 5.3oz Traditional Skyr Y...,0.0115,0.0165,0.0105,0.913043,55.335968,0.010310,11.310250,0.993352
4,(Blueberries 1 Pint),(Dole Bananas 2 lb Bag),0.0290,0.0715,0.0105,0.362069,5.063902,0.008426,1.455486,0.826492
5,(Dole Bananas 2 lb Bag),(Blueberries 1 Pint),0.0715,0.0290,0.0105,0.146853,5.063902,0.008426,1.138139,0.864323
6,(Strawberries 1 lb),(Blueberries 1 Pint),0.0505,0.0290,0.0120,0.237624,8.193923,0.010535,1.273649,0.924653
7,(Blueberries 1 Pint),(Strawberries 1 lb),0.0290,0.0505,0.0120,0.413793,8.193923,0.010535,1.619735,0.904180
8,(Fresh Brand Cut and Peeled Baby Carrots 16 oz),(Dole Bananas 2 lb Bag),0.0395,0.0715,0.0110,0.278481,3.894839,0.008176,1.286868,0.773816
9,(Dole Bananas 2 lb Bag),(Fresh Brand Cut and Peeled Baby Carrots 16 oz),0.0715,0.0395,0.0110,0.153846,3.894839,0.008176,1.135136,0.800485


#### Interpretion:
Support: **1%** of the transactions contain both antecedents and consequents.\
Confidence: **86.9%** of the customers who purchased all the products in antecedents also purchased all the items in consequents.\
Lift: Customers who purchased antecedents items are **82.8** times more likely to purchase the items in consequents.

#### Conclusion:
The consequents items with high confidence and lift can be recommended to those customers who have purchased the corresponding antecedents.


### Exporting the association rules and recommendations to the database

Preparing the data for creating recommendations table.

In [98]:
# Convert the frozensets antecedents and consequents to string type
rules['antecedents'] = rules['antecedents'].apply(lambda x: str(x))
rules['consequents'] = rules['consequents'].apply(lambda x: str(x))



In [109]:
# Remove the unnecessary characters from the antecedents and consequents features to match their values with product_name feature values
rules['antecedents'] = rules['antecedents'].apply(lambda x: x.strip('frozenset(){}'))
rules['consequents'] = rules['consequents'].apply(lambda x: x.strip('frozenset(){})'))
rules['antecedents'] = rules['antecedents'].apply(lambda x: x.strip("'"))
rules['consequents'] = rules['consequents'].apply(lambda x: x.strip("'"))


In [112]:
# Filter the main DataFrame for rows where product_name is in antecedents
filtered_df = df[df['product_name'].isin(rules['antecedents'])]

# Merge with rules to get corresponding recommendations
recommendations = filtered_df.merge(
    rules[['antecedents', 'consequents']],
    left_on='product_name',
    right_on='antecedents',
    how='inner'
)[['survey_response_id', 'product_name', 'consequents']]

# Rename columns for better understanding
recommendations.columns = ['survey_response_id', 'product_name', 'recommendations']

# Drop duplicates
recommendations = recommendations.drop_duplicates()

# Display the updated DataFrame
recommendations


,survey_response_id,product_name,recommendations
0,R_0jFt3K3i4558lbj,Fresh Brand Cut and Peeled Baby Carrots 16 oz,Dole Bananas 2 lb Bag
1,R_10JcbruZLDf9fy2,Fresh Brand Cut and Peeled Baby Carrots 16 oz,Dole Bananas 2 lb Bag
3,R_1Cew4e6dmKZSryU,Fresh Brand Cut and Peeled Baby Carrots 16 oz,Dole Bananas 2 lb Bag
4,R_1E0ZZlhukNsNc7w,Fresh Brand Cut and Peeled Baby Carrots 16 oz,Dole Bananas 2 lb Bag
6,R_1E3TJBNJVtr3tcD,Fresh Brand Cut and Peeled Baby Carrots 16 oz,Dole Bananas 2 lb Bag
...,...,...,...
2777,R_bJI8lmXMZWL6oql,Icelandic Provisions 5.3oz Traditional Skyr Yo...,Icelandic Provisions 5.3oz Traditional Skyr Yo...
2778,R_bJI8lmXMZWL6oql,Icelandic Provisions 5.3oz Traditional Skyr Yo...,"""Tate's Bake Shop Cookies-Chocolate Chip 7 Ounce"""
2782,R_r3IM83Ecq7eY7QJ,Icelandic Provisions 5.3oz Traditional Skyr Yo...,365 by Whole Foods Market Pretzel Twists Mini ...
2783,R_r3IM83Ecq7eY7QJ,Icelandic Provisions 5.3oz Traditional Skyr Yo...,Icelandic Provisions 5.3oz Traditional Skyr Yo...


Finally, we export the association rules and recommendations to our MySQL db using the 'to_sql' method and the connection we created previously.

In [113]:
# Save to SQL
rules.to_sql('association_rules', con=connection, if_exists='replace', index=False)
recommendations.to_sql('recommendations', con=connection, if_exists='replace', index=False)

806